In [2]:
print("OK")

OK


In [3]:
%pwd

'/Users/pravalika/Documents/Projects/Medicalbot/Medical-bot/research'

In [4]:
import os
os.chdir("../")

In [5]:
#as we are using Pdf we need pdf loader
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data="data/")


In [8]:
#extracted_data

In [9]:
#Now to split data into text chunks 
def text_splitter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    texts = text_splitter.split_documents(extracted_data)

    return texts

In [10]:
texts = text_splitter(extracted_data)
print("Length of Text Chunks",len(texts))

Length of Text Chunks 7686


In [155]:
#texts

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
pip install -U sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [14]:
#Now we need an embedder so we download one from huggingface 
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

/var/folders/gk/wx1wrfk57jd319j1tz_s4clh0000gn/T/ipykernel_40167/1535316010.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
#query_result

In [114]:
from dotenv import load_dotenv
load_dotenv()

True

In [115]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [21]:
pip install pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [117]:
import os 
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['MISTRAL_API_KEY'] = MISTRAL_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [156]:
import sys
print(sys.executable)

/opt/anaconda3/envs/medibot/bin/python


In [27]:
pip install accelerator

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'accelerator' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'accelerator'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for accelerator: filename=accelerator-2024.9.13-cp310-cp310-macosx_11_0_arm64.whl size=593059 sha256=9e38b4155af346458679461b6f142077534f3448a3644ad26fd645248164d25d
  Stored in directory: /Users/pravalika/Library/Caches/pip/wheels/f0/bf/f5/70d3ab3189f39bc805a81edc4bb4bac8fa31ad35dede4f4ef1
Successfully built accelerator
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [accelerator] [accelerator]
Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install -qU langchain-pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [24]:
# Import PineconeVectorStore from langchain.vectorstores (updated import path)
from langchain.vectorstores import Pinecone

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

In [28]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-dnsz952.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [29]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts,
    index_name=index_name,
    embedding=embeddings, 
)

In [30]:
#Load Existing Index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [31]:
docsearch

In [32]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [99]:
retrieved_docs = retriever.invoke("What is gastrografin swallow?")

In [100]:
retrieved_docs

[Document(id='7cada361-d02b-4629-af5a-6486101d7ff8', metadata={'creationdate': '2024-01-10T14:42:32+00:00', 'creator': 'PDF24', 'moddate': '2024-07-04T19:00:36+00:00', 'page': 415.0, 'page_label': '416', 'producer': 'iLovePDF', 'source': 'data/_Step_Up_to_MRCP_Review_Note_for_Part_I_&_Part_II_Middle_East_&.pdf', 'total_pages': 2230.0}, page_content="Gastroenterology\n&\nHepatology\nBarium\nswallow\n-grossly\ndilated\nfilled\noesophagus\nwith\na\ntight\nstricture\nat\nthe\ngastroesophageal\n)unction\nresulting\nin\na\n'bird\n'sbeak\n'appearance\n.Tertiary\ncontractions\ngive\nrise\nto\na\ncorkscrew\nappearance\notthe\noesophagus\nThe\nmost\nappropriate\ninitial\ninvestigation\nof\na\nhigh\ndysphagia\nIs\na\nbarium\nswallow\n,which\nIdentifies\nthe\nsite\nof\npathology\nand\nforewarns\nof\npitfalls\nsuch\nas\na\npharyngeal\npouch\n,whlcli\nif\nunidentified\ncan\nIncrease\nthe\nrisk\nof\nperforation\nat"),
 Document(id='28b54127-e361-42f7-8940-21599ccf85f8', metadata={'creationdate': '202

In [113]:
pip install groq langchain langchain-core langchain-community


Note: you may need to restart the kernel to use updated packages.


In [139]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama3-8b-8192",  # or "mixtral-8x7b-32768"
    temperature=0.5,
    max_tokens=1500
)


In [140]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
     "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise and precise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


In [141]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Combine retriever and QA into a full RAG chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [151]:
query = "what are the features and causes of Acute pancreatitis?"
response = rag_chain.invoke({"input": query})

print("Answer:\n", response["answer"])

Answer:
 Acute pancreatitis is usually due to alcohol or gallstones. Features include severe epigastric pain that may radiate to the back, vomiting, epigastric tenderness, ileus, and low-grade fever.


In [154]:
query = "Explain everything about Diverticulitis and where is it in book page number?"
response = rag_chain.invoke({"input": query})

print("Answer:\n", response["answer"])

Answer:
 I can help you with that! According to the provided context, Diverticulitis is a condition where one of the diverticular becomes infected. The classical presentation includes left iliac fossa pain and tenderness, anorexia, nausea and vomiting, diarrhea, and features of infection such as pyrexia, raised WBC and CRP.

The management of diverticulitis varies depending on the severity of the episode. Mild attacks can be treated with oral antibiotics, while more significant episodes require hospitalization, where patients are made nil by mouth, given intravenous fluids and intravenous antibiotics.

Complications of diverticulitis include symptomatic diverticulosis, which is a more accurate term for diverticula being present. Risk factors for diverticulitis include increasing age and low-fiber diet.

Diverticulitis can present in different ways, including painful diverticular disease, which is characterized by altered bowel habit, colicky left-sided abdominal pain. A high-fiber diet

In [132]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know.
